<a href="https://colab.research.google.com/github/NishchithRao-hub/ansible_se_google_doc/blob/main/Ansible_SE_Assessmnent_Nishchith_Rao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.2
    Uninstalling google-auth-oauthlib-1.2.2:
      Successfully uninstalled google-auth-oauthlib-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.41.1 which is incompatible.


In [14]:
from typing import List

def find_substrings(s: str, substring: str) -> List[int]:
  position = []
  start = 0
  while True:
    index = s.find(substring, start)
    if index == -1:
      break
    position.append(index)
    start = index + len(substring)
  return position

In [15]:
MARKDOWN = """

# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [16]:
import re
from typing import Tuple

def build_markdown(md: str) -> Tuple[str, List[dict]]:

    lines = md.splitlines()
    output = []
    elements = []
    for l in lines:
        stripped = l.rstrip()
        if stripped.startswith('# '):
            text = stripped[2:].strip()
            markers = (f"<<H1_START>>{text}<<H1_END>>")
            output.append(markers)
            elements.append({"type":"h1","text":text})
        elif stripped.startswith('## '):
            text = stripped[3:].strip()
            markers = (f"<<H2_START>>{text}<<H2_END>>")
            output.append(markers)
            elements.append({"type":"h2","text":text})
        elif stripped.startswith('### '):
            text = stripped[4:].strip()
            markers = (f"<<H3_START>>{text}<<H3_END>>")
            output.append(markers)
            elements.append({"type":"h3","text":text})
        elif re.match(r'^\s*-\s\[\s\]\s', stripped):
            text = re.sub(r'^\s*-\s\[\s\]\s', '', stripped)
            markers = (f"<<CB_START>>{text}<<CB_END>>")
            output.append(markers)
            elements.append({"type":"checkbox","text":text})
        elif stripped == '---':
            output.append("<<HR>>")
            elements.append({"type":"hr","text":"---"})
        elif stripped.strip().startswith(('- ', '* ')):
            leading_ws = re.match(r'^(\s*)', l).group(1)
            text = re.sub(r'^\s*[-*]\s+', '', stripped)
            output.append(leading_ws + text)
            elements.append({"type":"para","text":text, "raw_line":l})
        elif stripped == '':
            output.append('')
        else:
            output.append(stripped)
            if stripped.lower().startswith('meeting recorded by:') or stripped.lower().startswith('duration:'):
                elements.append({"type":"footer","text":stripped})
            else:
                elements.append({"type":"para","text":stripped})
    plain_text = "\n".join(output) + "\n"
    return plain_text, elements

plain_text, elements_preview = build_markdown(MARKDOWN)

In [17]:
print(plain_text[:500])



<<H1_START>>Product Team Sync - May 15, 2023<<H1_END>>

<<H2_START>>Attendees<<H2_END>>
Sarah Chen (Product Lead)
Mike Johnson (Engineering)
Anna Smith (Design)
David Park (QA)

<<H2_START>>Agenda<<H2_END>>

<<H3_START>>1. Sprint Review<<H3_END>>
Completed Features
  User authentication flow
  Dashboard redesign
  Performance optimization
    Reduced load time by 40%
    Implemented caching solution
Pending Items
  Mobile responsive fixes
  Beta testing feedback integration

<<H3_START>>2. Cur


In [18]:
# Referred ChatGPT for this.

from google.colab import auth
import google.auth
from googleapiclient.discovery import build

auth.authenticate_user()
credential, _ = google.auth.default(scopes=['https://www.googleapis.com/auth/documents','https://www.googleapis.com/auth/drive'])
docs_service = build('docs', 'v1', credentials=credential)
drive_service = build('drive', 'v3', credentials=credential)
print("Completed authentication and created Docs & Drive service clients.")

Completed authentication and created Docs & Drive service clients.


In [19]:
from googleapiclient.errors import HttpError

def create_doc(title: str):
  try:
    body = {'title': title}
    doc = docs_service.documents().create(body=body).execute()
    print(f"Created document with ID: {doc.get('documentId')}")
    return doc
  except HttpError as err:
    print("Error creating document:", err)
    raise

doc = create_doc("Product Team Sync")
doc_id = doc.get('documentId')

Created document with ID: 1KB0YIoyf0paT0bzSGGmQ6oggXdZ_a87vuAyk8VnacSE


In [20]:
def insert_text_to_doc(document_id: str, text: str):
    requests = [
        {
            'insertText': {
                'location': {'index': 1},
                'text': text
            }
        }
    ]
    resp = docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    return resp

resp = insert_text_to_doc(doc_id, plain_text)

In [21]:
def utf16(s: str) -> int:
    return len(s.encode('utf-16-le')) // 2

In [22]:
import json

def build_format(text: str):
    requests = []
    base_offset = 1

    for level in (1,2,3):
        start = f"<<H{level}_START>>"
        end = f"<<H{level}_END>>"
        position = find_substrings(text, start)
        for s in position:
            start_content = base_offset + utf16(text[:s + len(start)])
            # find end
            curr_end = text.find(end, s + len(start))
            end_content = base_offset + utf16(text[:curr_end])
            style = {1: 'HEADING_1', 2: 'HEADING_2', 3: 'HEADING_3'}[level]
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': start_content, 'endIndex': end_content},
                    'paragraphStyle': {'namedStyleType': style},
                    'fields': 'namedStyleType'
                }
            })

    cb_start = find_substrings(text, "<<CB_START>>")
    for s in cb_start:
        start_content = base_offset + utf16(text[:s + len("<<CB_START>>")])
        curr_end = text.find("<<CB_END>>", s + len("<<CB_START>>"))
        end_content = base_offset + utf16(text[:curr_end])
        requests.append({
            'createParagraphBullets': {
                'range': {'startIndex': start_content, 'endIndex': end_content},
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })
        substring = text[s + len("<<CB_START>>"): curr_end]
        for m in re.finditer(r'@([A-Za-z0-9_.-]+)', substring):
            mention_start = s + len("<<CB_START>>") + m.start()
            mention_end = s + len("<<CB_START>>") + m.end()
            abs_start = base_offset + utf16(text[:mention_start])
            abs_end = base_offset + utf16(text[:mention_end])
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': abs_start, 'endIndex': abs_end},
                    'textStyle': {'bold': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0.12, 'green': 0.34, 'blue': 0.55}}}},
                    'fields': 'bold,foregroundColor'
                }
            })

    for footer in ('Meeting recorded by:', 'Duration:'):
        for pos in find_substrings(text, footer):
            line_end = text.find('\n', pos)
            if line_end == -1:
                line_end = len(text)
            abs_start = base_offset + utf16(text[:pos])
            abs_end = base_offset + utf16(text[:line_end])
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': abs_start, 'endIndex': abs_end},
                    'textStyle': {'italic': True, 'fontSize': {'magnitude': 10, 'unit': 'PT'}, 'foregroundColor': {'color': {'rgbColor': {'red': 0.35, 'green': 0.35, 'blue': 0.35}}}},
                    'fields': 'italic,fontSize,foregroundColor'
                }
            })

    marker_list = ["<<H1_START>>","<<H1_END>>","<<H2_START>>","<<H2_END>>","<<H3_START>>","<<H3_END>>",
                   "<<CB_START>>","<<CB_END>>","<<HR>>"]
    for mk in marker_list:
        requests.append({
            'replaceAllText': {
                'containsText': {'text': mk, 'matchCase': True},
                'replaceText': ''
            }
        })

    hr_replace = {
        'replaceAllText': {
            'containsText': {'text': '<<HR>>', 'matchCase': True},
            'replaceText': '\n\u2014\u2014\u2014\u2014\u2014\n'
        }
    }
    requests.insert(0, hr_replace)

    return requests

format_requests = build_format(plain_text)
print(f"These are the {len(format_requests)} formatting requests:")
for r in format_requests[:8]:
    print(json.dumps(r, indent=2))

These are the 29 formatting requests:
{
  "replaceAllText": {
    "containsText": {
      "text": "<<HR>>",
      "matchCase": true
    },
    "replaceText": "\n\u2014\u2014\u2014\u2014\u2014\n"
  }
}
{
  "updateParagraphStyle": {
    "range": {
      "startIndex": 15,
      "endIndex": 47
    },
    "paragraphStyle": {
      "namedStyleType": "HEADING_1"
    },
    "fields": "namedStyleType"
  }
}
{
  "updateParagraphStyle": {
    "range": {
      "startIndex": 71,
      "endIndex": 80
    },
    "paragraphStyle": {
      "namedStyleType": "HEADING_2"
    },
    "fields": "namedStyleType"
  }
}
{
  "updateParagraphStyle": {
    "range": {
      "startIndex": 193,
      "endIndex": 199
    },
    "paragraphStyle": {
      "namedStyleType": "HEADING_2"
    },
    "fields": "namedStyleType"
  }
}
{
  "updateParagraphStyle": {
    "range": {
      "startIndex": 880,
      "endIndex": 892
    },
    "paragraphStyle": {
      "namedStyleType": "HEADING_2"
    },
    "fields": "namedStyleTyp

In [23]:
try:
    batch_resp = docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': format_requests}).execute()
    print("Formatting request applied.")
except HttpError as e:
    print("Error applying formatting:", e)
    raise

Formatting request applied.


In [24]:
def create_shareable_link(file_id: str, role='writer'):
    try:
        drive_service.permissions().create(
            fileId=file_id,
            body={'type': 'anyone', 'role': role},
            fields='id'
        ).execute()
        meta = drive_service.files().get(fileId=file_id, fields='webViewLink').execute()
        return meta.get('webViewLink')
    except HttpError as e:
        print("Permission error or sharing disabled:", e)
        raise

link = create_shareable_link(doc_id, role='writer')

In [25]:
print("Document ID:", doc_id)
print("This is the link:", link)

Document ID: 1KB0YIoyf0paT0bzSGGmQ6oggXdZ_a87vuAyk8VnacSE
This is the link: https://docs.google.com/document/d/1KB0YIoyf0paT0bzSGGmQ6oggXdZ_a87vuAyk8VnacSE/edit?usp=drivesdk
